<a href="https://colab.research.google.com/github/NiceNewton/NiceNewton/blob/main/livesalemodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fusion Module (FM)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LeakyReLU, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
import numpy as np

# Sample advertising data (Batch Size: 16, Time Steps: 10, Features: 5)
np.random.seed(42)
ad_data = np.random.rand(16, 10, 5)  # Simulatxing ad features over time

# Define Fusion Module (FM)
def build_fusion_module():
    model = Sequential([
        Conv1D(filters=16, kernel_size=4, padding='same', strides=1, input_shape=(10, 5)),  # 1D Convolution
        LeakyReLU(alpha=0.1),  # LeakyReLU Activation
        MaxPooling1D(pool_size=2),  # MaxPooling Layer
        Flatten(),  # Flatten Layer
        Dense(32, activation='relu'),  # Fully Connected Layer (MLP)
        Dense(16, activation='relu')   # Final Feature Representation
    ])
    return model

# Build the model
fusion_module = build_fusion_module()

# Forward pass with sample data
ad_features = fusion_module(ad_data)

# Print Output Shape
print("Output Shape of FM:", ad_features.shape)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Output Shape of FM: (16, 16)


Interaction Module I (IM-I)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, MultiHeadAttention, Concatenate, Reshape, Layer

# Define Interaction Module I (IM-I)
class InteractionModuleI(tf.keras.Model):
    def __init__(self, feature_dim, num_heads=4):
        super(InteractionModuleI, self).__init__()
        self.multi_head_attention = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.feature_dense = Dense(feature_dim, activation='tanh')  # Feature Importance Layer
        self.reshape = Reshape((1, feature_dim)) # Reshape layer added here

    def call(self, ad_features, live_features):
        # Reshape the input tensors to add a sequence length dimension of 1
        # This is necessary for the MultiHeadAttention layer
        ad_features = self.reshape(ad_features) # Applying reshape here
        live_features = self.reshape(live_features) # Applying reshape here

        # Concatenate Advertising Features (g_t) and Live Features (L_t)
        x_t = Concatenate(axis=1)([ad_features, live_features])  # Concatenate along the sequence length dimension

        # Apply Multi-Head Attention
        attention_output, _ = self.multi_head_attention(x_t, x_t, return_attention_scores=True)

        # Instead of removing dimension 1, which has size 2 due to concatenation,
        # we should take the mean or select a specific element along this dimension.
        # Here, we take the mean along axis=1 to combine information from both features:
        attention_output = tf.reduce_mean(attention_output, axis=1)

        # Compute Feature Importance Weights
        weighted_features = self.feature_dense(attention_output)

        return weighted_features

# Example Input Data (Batch Size: 16, Features: 32)
ad_features = tf.random.normal((16, 32))  # Simulated Ad Features
live_features = tf.random.normal((16, 32))  # Simulated Live-Broadcast Features

# Initialize and Test IM-I
interaction_module = InteractionModuleI(feature_dim=32)
output_features = interaction_module(ad_features, live_features)

print("Output Shape of IM-I:", output_features.shape)

Output Shape of IM-I: (16, 32)


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten, Concatenate, Dropout
from tensorflow.keras.models import Model

# Define TCN block
def TCN_block(input_shape, filters=64, kernel_size=3, dilation_rate=2):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding="causal", activation="relu")(inputs)
    x = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding="causal", activation="relu")(x)
    return Model(inputs, x, name="TCN_Block")

# Define TIM-II (TCN + MLP Fusion)
def TIM_II(tcn_input_shape, extra_feature_dim):
    # TCN block
    tcn_input = Input(shape=tcn_input_shape, name="TCN_Input")
    tcn_output = TCN_block(tcn_input_shape)(tcn_input)

    # MLP to transform TCN output
    mlp_output = Dense(128, activation="relu")(tcn_output)
    mlp_output = Flatten()(mlp_output)

    # Extra Features Input
    extra_features = Input(shape=(extra_feature_dim,), name="Extra_Features")

    # Concatenate TCN processed output with extra features
    fused = Concatenate()([mlp_output, extra_features])

    # Final Dense Layers for Prediction
    x = Dense(128, activation="relu")(fused)
    x = Dropout(0.2)(x)
    x = Dense(64, activation="relu")(x)
    output = Dense(1, activation="sigmoid", name="Prediction")(x)  # Adjust activation based on task

    # Define Model
    model = Model(inputs=[tcn_input, extra_features], outputs=output, name="TIM_II_Model")
    return model

# Example usage
tcn_shape = (30, 1)  # 30 time steps, 1 feature per time step
extra_feature_dim = 10  # Example: 10 additional features

model = TIM_II(tcn_shape, extra_feature_dim)
model.summary()


Model: "TIM_II_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ TCN_Input (InputLayer)    │ (None, 30, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ TCN_Block (Functional)    │ (None, 30, 64)         │         12,608 │ TCN_Input[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 30, 128)        │          8,320 │ TCN_Block[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 3840)           │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Extra_Features            │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 3850)           │              0 │ flatten[0][0],         │
│                           │                        │                │ Extra_Features[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │        492,928 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │          8,256 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Prediction (Dense)        │ (None, 1)              │             65 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 522,177 (1.99 MB)

 Trainable params: 522,177 (1.99 MB)

 Non-trainable params: 0 (0.00 B)